# Logistic Regression
## MaSSP 2017, Computer Science

__Chuẩn bị: Xếp Châu__

Trong lab này, chúng ta sẽ học cách dùng logistic regression để phân loại 2 nhóm chữ số viết tay "0" và "1".

Cách viết code trong __Phần 1__ dựa vào công thức đã học trong bài giảng về logistic regression, đặc biệt dành cho trường hợp có __2 nhóm__ cần phân loại.

__Phần 2__ là bài tập thay đổi code về cách tính hàm cost function, chiều của tensor, để áp dụng cho phân loại tất cả __10 nhóm__ chữ số viết tay từ "0" đến "9". Phần này nên được làm sau khi đã học về $softmax\ regression$ trong bài giảng tiếp theo - Deep Neural Network.

# 0. Khái quát bài toán và kho dữ liệu MNIST

Dựa theo http://neuralnetworksanddeeplearning.com/chap1.html#a_simple_network_to_classify_handwritten_digits.

## 0.1 Khái quát vấn đề
Nhận dạng chữ viết và chữ số viết tay có rất ứng dụng to lớn trong cuộc sống, ví dụ như chuyển thông tin của các đơn điền bằng tay vào máy tính (tưởng tượng đến số lượng đơn chúng ta phải điền tay hàng năm!).

Mặc dù kĩ thuật OCR (_Optical Character Recognition_) có thể chuyển hóa hình ảnh của tài liệu đánh máy thành chữ và lưu trên máy tính, kĩ thuật này gặp khó khăn trong nhận dạng chữ viết tay vì chữ viết của mỗi người có những đặc điểm riêng biệt.

Chúng ta có thể phân tích vấn đề nhận dạng một số thành 2 vấn đề nhỏ hơn. Đầu tiên, khi cho hình ảnh của một dãy số viết tay (ví dụ như số điện thoại), chúng ta cần chia hình ảnh này ra thành những hình ảnh chứa từng con số. Ví dụ như với dãy số này:

![sequence_of_digits](http://neuralnetworksanddeeplearning.com/images/digits.png "a sequence of digits")

... ta sẽ biến dãy này thành các hình ảnh sau:

![sequence_of_images](http://neuralnetworksanddeeplearning.com/images/digits_separate.png "a sequence of images")

Sau đó, chúng ta sẽ viết chương trình để phân loại từng chữ số. Đây cũng là vấn đề chúng ta sẽ giải quyết trong lab này.

Hình ảnh các chữ số viết tay được lấy từ kho dữ liệu __MNIST__.

## 0.2 Tìm hiểu kho dữ liệu MNIST
Một số thông tin về kho dữ liệu [MNIST data set](http://yann.lecun.com/exdb/mnist/): 
- Bao gồm __70000__ ảnh cỡ 28x28 pixel của các chữ số viết tay được scan lên
    * __55000__ hình ảnh đen trắng cho __training data__
    * __5000__ cho __validation data__
    * __10000__ cho __test data__
- Mỗi pixel mang giá trị từ 0.0 (đen) đến 1.0 (trắng), các giá trị ở giữa 0.0 và 1.0 thể hiện màu xám có độ đen giảm dần

__Training data__ được dùng trong quá trình học, __test data__ được dùng để kiểm tra độ chính xác của model <i>sau</i> quá trình học.

Tại sao chúng ta lại cần đến __validation data__? Thay vì học từ 55000 ảnh trong training set, tại sao không dùng tất cả 60000 ảnh?

_Trong_ quá trình học, thuật toán sẽ sử dụng __validation data__ để kiểm tra xem quá trình học có đang diễn ra tốt không.

Validation set giúp tránh hiện tượng <i>overfitting</i> - nếu chỉ sử dụng training data để đánh giá model, ta có thể sẽ thu được một model làm rất tốt trên training set, nhưng lại cho kết quả thấp khi thực hiện phân loại trên data nó chưa từng nhìn thấy như test set.

# 1. Logistic Regression cho 2 nhóm

## 1.1 Tóm tắt
Đầu tiên hãy ôn lại các bước trong thuật toán logistic regression.

<b>0. Lấy dữ liệu cho training set và test set</b>

Sử dụng MNIST dataset có thể download qua thư viện TensorFlow, lọc ra các hình ảnh của chữ số "0" và "1".

<img src="../../images/Logistic_Regression/zeros_and_ones.png" />

<b>1. Tính output</b>

Trong lab ta sẽ sử dụng kí hiệu hơi khác trong bài giảng. Cụ thể ta thay $\theta$ bằng $W$, $\theta_0$ bằng $b$. 

Các giá trị ban đầu của $W$ và $b$ gán bằng 0.
$$z = x* W + b$$

$$y = sigmoid(z) = \frac{1}{1 + exp(-z)}$$  

<b>2. Dùng cost function, so sánh output với label chuẩn</b>

$$cost = -\frac{1}{m}(y\_correct*log(y) + (1-y\_correct)*log(1-y))$$

<b>3. Dùng Gradient descent để điều chỉnh $W$ và $b$ sao cho $cost$ nhỏ nhất</b>

Sử dụng hàm $GradientDescentOptimizer$ của TensorFlow!

<b>4. Lặp lại các bước 1-3 cho đến khi đạt được độ chính xác mong muốn</b>

* Training error: tỉ lệ phân loại sai trong training set
* Validation error: tỉ lệ phân loại sai trong validation set
* Test error: tỉ lệ phân loại sai trong test set

## 1.2 Tải dữ liệu
Bắt đầu với việc tải những thư viện quen thuộc.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

Tiếp đó là tải dữ liệu MNIST nhờ thư viện TensorFlow.

Chi tiết: https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/tutorials/mnist 

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

<img style="float: left" src="../../images/Logistic_Regression/read_data_sets.png" />
Hàm $read\_data\_sets$ từ $input\_data$ dùng để tải dữ liệu, tùy vào arguments mà dạng của dữ liệu sẽ khác nhau. 

Ví dụ nếu ta đặt $one\_hot=True$, thì mỗi label nhận được sẽ có chiều 1x10, với giá trị 1 tại vị trí tương ứng với chữ số đó, và 0 cho các vị trí còn lại. 

Ví dụ, số "1" sẽ được biếu diễn bởi array sau:
$$[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]$$

Còn với $one\_hot=False$, mỗi label sẽ có giá trị là một số nguyên ứng với chữ số nó biểu thị: $label=1$ cho hình ảnh của chữ số 1, v.v.

Trong phần 1, chúng ta chỉ làm việc với 2 chữ số "0" và "1", nên chỉ cần label có giá trị 0 và 1. 

Khi gọi $read\_data\_sets$, chúng ta thậm chí không cần cung cấp giá trị $one\_hot=False$, vì $False$ là giá trị mặc định của $one\_hot$ trong hàm $read\_data\_sets$ rồi.

Chi tiết: https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/tutorials/mnist 

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/")
mnist

Dữ liệu tải về được chứa trong thư mục MNIST_data/ và được load vào biến $mnist$.

$mnist$ chứa 3 Dataset: train, validation, và test. 
>Tìm hiểu thêm về TensorFLow's Dataset class:
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/learn/python/learn/datasets/mnist.py 

Ta sẽ gán 3 dataset này cho các biến $train\_data$, $validation\_data$, và $test\_data$.

In [ ]:
(train_data, validation_data, test_data) = (mnist.train, mnist.validation, mnist.test)

Mỗi Dataset này đều có $images$ và $labels$, chứa các hình ảnh 28x28 và loại tương ứng (cũng là các số từ 0-9).

__Checkpoint 1__: Hãy tìm số hình ảnh trong của mỗi Dataset bằng code.

In [ ]:
# code


__Checkpoint 2__: Hãy lọc ra hình ảnh và label tương ứng của các chữ số "0" và "1" trong training, validation và test set.

In [ ]:
#train_images = 
#train_labels = 

#test_images = 
#test_labels = 

Có bao nhiêu hình ảnh các số 0 và 1 trong từng set? Hãy plot hình ảnh đầu tiên trong training set và kiểm tra xem label tương ứng có phù hợp với hình ảnh thu được hay không.

In [ ]:
# code


In [ ]:
# Skip for now: Các thay đổi khác trên dữ liệu nếu cần thiết cho phần sau


## 1.3 Viết chương trình phân loại

### Inputs
__Checkpoint 3__: Hãy định nghĩa các tensors sau trong scope "Inputs":
- $x$: chứa tất cả các ảnh trong training set,  type là $float32$
- $y\_correct$: chứa labels của các ảnh trong training set, type là $float32$
- $W$: chứa các hệ số tương ứng với từng điểm ảnh, $tf.Variable$
- $b$: bias hay $\theta_0$ trong bài giảng, $tf.Variable$

Lưu ý: Thay vì định nghĩa $x$ và $y\_correct$ là $tf.constant$ với giá trị ban đầu là các array rất lớn, ta nên sử dụng $tf.placeholder$ - một cách để "đặt chỗ" $x$ và $y\_correct$ mà không cần cung cấp array ngay. 

Chỉ đến khi chạy trong $session$, ta mới cung cấp những array này (có thể lấy từ $train\_data$, $validation\_data$, hoặc $test\_data$).

In [ ]:
with tf.name_scope("Inputs") as scope:
    #x = tf.placeholder(...)
    #y_correct = tf.placeholder(...)
    #W = tf.Variable(...)
    #b = tf.Variable(...)

### Hypothesis function
__Checkpoint 4__: Hãy viết công thức tính $y$ dựa vào $W$ và $b$.

$$z = x* W + b$$

$$y = sigmoid(z) = \frac{1}{1 + exp(-z)}$$

Lưu ý $y$ không phải chỉ bao gồm các số 0 và 1 - tương ứng với label "0" và "1", mà là các giá trị số thực từ 0 đến 1. Giá trị càng gần 1 thì khả năng label là "1" càng cao, và giá trị càng gần 0 thì khả năng label là "1" càng thấp (hay nói cách khác khả năng label là "0" càng cao).

In [ ]:
with tf.name_scope("Outputs") as scope:
    #y = ...

### Cost function
__Checkpoint 5__: Hãy viết công thức tính $cost$ thể hiện sai số giữa kết quả dự đoán $y$ và label thật $y\_corect$. Lưu ý rằng giá trị thu được từ operation $cost$ không phải là một array như $y$ hay $y\_correct$ mà là một số thực.

$$cost = -\frac{1}{m}(y\_correct*log(y) + (1-y\_correct)*log(1-y))$$

In [ ]:
with tf.name_scope("Cost") as scope:
    #cost = ...

### Gradient Descent
__Checkpoint 6__: Hãy dùng $GradientDescentOptimizer()$ của TensorFlow để viết operation $train\_step$ tương tự như trong lab đầu tiên về TensorFlow.

In [ ]:
with tf.name_scope('Train') as scope:
    #train_step = ...

### Evaluate model
Trong quá trình training, chúng ta cần biết độ chính xác của phân loại để điều chỉnh thuật toán và tham số cho phù hợp. Như đã nói ở trên, các giá trị trong $y$ dao động từ 0 đến 1, ta cần quyết định label dự đoán cho mỗi ảnh là 0 hoặc 1 dựa vào các giá trị này.

__Checkpoint 7__: Hãy viết một operation có thể tính được độ chính xác của model khi phân loại dataset (dựa vào $y$ của model và label thật $y\_correct$ của dataset).

In [ ]:
with tf.name_scope("Accuracy") as scope:
    #...

### Run session
Vậy là chúng ta đã sẵn sàng chạy thuật toán logistic regression!
Để kiểm tra phần code trên không mắc các lỗi thường gặp, hãy chạy thử 1 bước $train\_step$ ở trên. Đừng quên tạo một $InteractiveSession$ và khởi tạo tất cả các biến trước khi chạy bất kì operation nào.

Với các tensor được khởi tạo bởi $tf.placeholder$, hãy nhớ cung cấp các giá trị thật cho chúng, ví dụ:

__Checkpoint 8__: Chạy thử một bước $train\_step$.

In [ ]:
# code


__Checkpoint 9__: Sau khi đảm bảo bước $train\_step$ chạy như ý, hãy thêm vào các $summary$ tương tự như lab đầu tiên về TensorFlow để theo dõi sự biến thiên của các biến trong TensorBoard, nhất là độ chính xác của thuật toán.

In [ ]:
# code


# 2. (Optional) Logistic Regression cho nhiều nhóm

## 2.1 Tóm tắt
Trong phần 1, $y_{(i)}$ thể hiện khả năng một hình ảnh $i$ có label là chữ số "1". Do chỉ có 2 nhóm, ta có thể coi $1-y_{(i)}$ là khả năng hình này có label là chữ số "0" và chỉ cần dùng một giá trị $y_{(i)}$ là có thể tính được cả 2 khả năng.

Một cách khác để phân loại 2 nhóm là tính hai giá trị $y_{(i)}^0$ và $y_{(i)}^1$ cho mỗi ảnh, giá trị nào cao hơn đồng nghĩa với việc label tương ứng có khả năng đúng cao hơn. Để các giá trị này tương ứng với xác suất, ta có thể dùng $softmax$ để làm $y_{(i)}^0, y_{(i)}^1>0$ và $y_{(i)}^0+y_{(i)}^1 = 1$.

Cách này có thể áp dụng cho phân loại nhiều hơn 2 nhóm. Ví dụ nếu có 3 nhóm, ta sẽ tính 3 giá trị $y_{(i)}^0$, $y_{(i)}^1$, và $y_{(i)}^2$, dùng hàm $softmax$ để biến đổi sao cho $y_{(i)}^0, y_{(i)}^1, y_{(i)}^2>0$ và $y_{(i)}^0+y_{(i)}^1+y_{(i)}^2 = 1$. Ảnh sẽ được phân loại theo giá trị cao nhất trong 3 giá trị này.

Lưu ý, khi sử dụng $softmax$, để tính dự đoán $y$, ta không cần áp dụng hàm sigmoid lên logit $z = W*x + b$ để tính $y$ như với trường hợp 2 nhóm. Hàm $softmax$ cần được áp dụng trực tiếp lên logit.

Chi tiết: https://en.m.wikipedia.org/wiki/Softmax_function.

## 2.2 Bài tập
Hai bài tập sau đây nên được làm sau khi đã học về $softmax$ trong bài giảng Deep Neural Network.
1. Dựa theo cách phân loại ở phần 1, viết thuật toán phân loại mới như đã mô tả ở trên để phân loại chữ số "0" và "1", với $y$ cho mỗi ảnh bao gồm 2 giá trị, xác suất của hình ảnh là số "0" và là số "1".
2. Thay đổi thuật toán ở câu trên một lần nữa để phân loại tất cả 10 nhóm các chữ số trong MNIST dataset. Độ chính xác thu được là bao nhiêu?